## 1. Importar Librerías

In [ ]:
import json
import os
import time
import sys
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

print(f"TensorFlow version: {tf.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 2. Configuración de Hiperparámetros

In [ ]:
# Configuración general
EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4

# Hiperparámetros del modelo
LEARNING_RATE = 0.001
FILTERS = 16
KERNEL_SIZE = 3
DENSE1 = 256
DENSE2 = 256
DENSE3 = 256
DROPOUT = 0.2

# Ruta de datos (ajustar según sea necesario)
DATA_DIR = "./gtsrb"  # Cambiar a la ruta de tus datos

print("Configuración:")
print(f"- Épocas: {EPOCHS}")
print(f"- Tamaño de imagen: {IMG_WIDTH}x{IMG_HEIGHT}")
print(f"- Categorías: {NUM_CATEGORIES}")
print(f"- Test size: {TEST_SIZE}")
print(f"- Learning rate: {LEARNING_RATE}")

## 3. Funciones de Carga de Datos

In [ ]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels = []
    
    for category in os.listdir(data_dir):
        category_dir = os.path.join(data_dir, str(category))
        if not os.path.isdir(category_dir):
            continue
            
        for filename in os.listdir(category_dir):
            img_path = os.path.join(category_dir, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))            
            images.append(img)
            labels.append(int(category))
    
    print(f"Carga de imágenes completada: {len(images)} imágenes.")
    return images, labels

## 4. Cargar y Preparar Datos

In [ ]:
# Cargar datos
images, labels = load_data(DATA_DIR)
print(f"Set de {len(images)} imágenes cargadas.")

# Convertir labels a categorical
labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CATEGORIES)

# Split en train y test
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE, random_state=42
)

print(f"\nDatos de entrenamiento: {x_train.shape}")
print(f"Labels de entrenamiento: {y_train.shape}")
print(f"Datos de prueba: {x_test.shape}")
print(f"Labels de prueba: {y_test.shape}")

### Visualizar algunas imágenes de ejemplo

In [ ]:
# Visualizar algunas imágenes del dataset
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Ejemplos de Imágenes de Entrenamiento', fontsize=16)

for i, ax in enumerate(axes.flat):
    # Convertir de BGR a RGB para visualización correcta
    img_rgb = cv2.cvtColor(x_train[i], cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    ax.set_title(f'Categoría: {np.argmax(y_train[i])}')
    ax.axis('off')

plt.tight_layout()
plt.show()

## 5. Definir el Modelo

In [ ]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    # Create a convolutional neural network
    model = tf.keras.models.Sequential([
        # Convolutional layer. It will learn filters using a kernel
        tf.keras.layers.Conv2D(
            filters=FILTERS, 
            kernel_size=(KERNEL_SIZE, KERNEL_SIZE), 
            activation="relu", 
            input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),

        # Flatten units
        tf.keras.layers.Flatten(),

        # Add hidden layers with dropout
        tf.keras.layers.Dense(DENSE1, activation="relu"),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Dense(DENSE2, activation="relu"),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Dense(DENSE3, activation="relu"),
        tf.keras.layers.Dropout(DROPOUT),

        # Add an output layer with output units for all categories
        tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    return model

### Crear y visualizar el modelo

In [ ]:
model = get_model()
model.summary()

## 6. Entrenar el Modelo

In [ ]:
# Entrenar el modelo
start_time = time.time()

history = model.fit(
    x_train, 
    y_train, 
    epochs=EPOCHS,
    validation_split=0.2,
    verbose=1
)

end_time = time.time()
training_time = end_time - start_time

print(f"\nEntrenamiento completado en {training_time:.2f} segundos ({training_time/60:.2f} minutos).")

## 7. Visualizar Resultados del Entrenamiento

In [ ]:
# Graficar accuracy y loss
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy')
if 'val_accuracy' in history.history:
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

# Loss
ax2.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 8. Evaluar el Modelo

In [ ]:
# Evaluate neural network performance
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)

print(f"\n{'='*50}")
print(f"RESULTADOS EN TEST SET")
print(f"{'='*50}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'='*50}")

## 9. Guardar el Modelo y Logs

In [ ]:
def save_log_data(history, training_time, test_accuracy, test_loss, log_file='training_log.json'):
    """
    Saves the log data from training and testing into a JSON file.
    
    Parameters:
    - history (tf.keras.callbacks.History): The history object containing the training history.
    - training_time (float): Time taken to train the model in seconds.
    - test_accuracy (float): The accuracy of the model on the test dataset.
    - test_loss (float): The loss of the model on the test dataset.
    - log_file (str): The path to the JSON file where the log data will be saved.
    
    Returns: None
    """
    # Cargar el contenido existente del archivo o inicializar un array vacío
    if os.path.exists(log_file):
        with open(log_file, 'r') as file:
            try:
                log_data = json.load(file)
            except json.JSONDecodeError:
                log_data = []
    else:
        log_data = []

    # Extraer los hiperparámetros y resultados del historial de entrenamiento
    new_log_entry = {
        "hyperparameters": {
            "learning_rate": LEARNING_RATE,
            "epochs": EPOCHS,
            "filters": FILTERS,
            "kernel_size": KERNEL_SIZE,
            "dense1": DENSE1,
            "dense2": DENSE2,
            "dense3": DENSE3,
            "dropout": DROPOUT
        },
        "results": {
            "train_accuracy": float(history.history['accuracy'][-1]),
            "train_loss": float(history.history['loss'][-1]),
            "test_accuracy": float(test_accuracy),
            "test_loss": float(test_loss),
            "training_time": round(training_time, 3)
        }
    }
    log_data.append(new_log_entry)

    # Escribir el array completo de nuevo en el archivo
    with open(log_file, 'w') as file:
        json.dump(log_data, file, indent=4)
    
    print(f"\nLogs guardados en: {log_file}")

In [ ]:
# Guardar el modelo
model_filename = "model.keras"
model.save(model_filename)
print(f"Modelo guardado en: {model_filename}")

# Guardar los logs
log_filename = "training_log_notebook.json"
save_log_data(history, training_time, test_accuracy, test_loss, log_filename)

## 10. Predicciones de Ejemplo

In [ ]:
# Hacer predicciones en algunas imágenes de prueba
num_examples = 10
predictions = model.predict(x_test[:num_examples])
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test[:num_examples], axis=1)

# Visualizar predicciones
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Predicciones del Modelo', fontsize=16)

for i, ax in enumerate(axes.flat):
    img_rgb = cv2.cvtColor(x_test[i], cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    
    color = 'green' if predicted_classes[i] == true_classes[i] else 'red'
    ax.set_title(f'Real: {true_classes[i]}\nPred: {predicted_classes[i]}', color=color)
    ax.axis('off')

plt.tight_layout()
plt.show()

# Calcular accuracy en estos ejemplos
correct = np.sum(predicted_classes == true_classes)
print(f"\nPrecisión en los {num_examples} ejemplos: {correct}/{num_examples} ({correct/num_examples*100:.1f}%)")

## 11. Análisis de Resultados

### Resumen de Métricas

In [ ]:
print("\n" + "="*60)
print("RESUMEN COMPLETO DEL ENTRENAMIENTO")
print("="*60)
print("\nHIPERPARÁMETROS:")
print(f"  - Learning Rate: {LEARNING_RATE}")
print(f"  - Épocas: {EPOCHS}")
print(f"  - Filtros Conv: {FILTERS}")
print(f"  - Kernel Size: {KERNEL_SIZE}x{KERNEL_SIZE}")
print(f"  - Dense Layers: {DENSE1}, {DENSE2}, {DENSE3}")
print(f"  - Dropout: {DROPOUT}")
print("\nRESULTADOS DE ENTRENAMIENTO:")
print(f"  - Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"  - Train Loss: {history.history['loss'][-1]:.4f}")
if 'val_accuracy' in history.history:
    print(f"  - Val Accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print(f"  - Val Loss: {history.history['val_loss'][-1]:.4f}")
print("\nRESULTADOS DE PRUEBA:")
print(f"  - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  - Test Loss: {test_loss:.4f}")
print(f"\nTiempo de entrenamiento: {training_time:.2f} segundos")
print("="*60)